---
title: Day4. CNN을 사용한 특징 기반 분류 v1.2 실습(Hands-On)
---

> [CNN](https://hal.science/hal-03926082/document)을 학습하고, [VGG](https://arxiv.org/abs/1409.1556)에 대해서 알아보자

- LeNet 이전의 분류 작업
    - `Pattern`을 `수식적`으로 작성 => 분류기에 해당 수식을 사용해서 분류를 진행
        - 예를 들어, 1) 입력 이미지가 주어지면 해당 입력에 대해 수학적인 방식으로 입력이미지의 특징들을 추출(면,선,코너 등의 정보), 2) 해당 특징들을 기반으로 적절한 특징들을 선별해서 해당 특징들의 벡터와 같은 추가적인 정보를 획득, 3) 최종적으로 이러한 정보들을 활용해 사물을 인식하는 과정을 거침
    - 한계점
        - 특징 추출에 많은 사전지식이 포함되어 있어야 함
        - 특징 추출 과정에서 이러한 특징들을 어떻게 특징을 추출하는지에 따라 성능이 좌우
        - 새로운 문제(새로운 특징이 추가적으로 필요하거나 기존과는 다른 패턴이 보이는 경우)에는 다시 시스템을 구축

- CNN(convolution neural network)
    - CNN은 작은 필터를 이용해 이미지로부터 특징을 추출해내는 방법
        - 기존 모델은 모든 픽셀에 대해 가중치를 갖고 있고, 전체를 특징으로 이용하기 때문에 학습에 사용된 데이터에 관해서는 완벽에 가까울 정도로 특징을 잡아낼 수 있는데, 특징 위치가 바뀌게 되면 무용지물
        - 반대로 합성곱은 커널을 이미지 안에서 이리저리 움직이며 특징을 추출, 따라서 볼 수 있는 시야는 좁아지는 대신 위치와 무관하게 특징을 잡아 낼 수 있음
        - 이미지 전체에 가중치를 두는 모델은 이미지 크기가 커지면 학습해야 하는 가중치 개수도 늘어나는데, CNN 커널 크기는 변화가 없음, 즉 커널을 사용하면 이미지 크기와 무관하게 학습해야 하는 가중치 개수가 같음
        - 학습할 가중치도 줄어들고, 특징의 위치에 대해 어느 정도 자유로워졌으니 두 마리 토끼를 다 잡은 것

- [`LeNet-5`](http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf)
    - `convolution neural network`를 활용
    - CNN은 기존의 방식과는 다르게 이미지를 일정한 지역적인 패턴을 이용하는 특수한 구조
    - 다중 신경망 네트워크는 복잡하고, 고차원적이고, 비선형 mapping을 이미지 인식 작업에서 여러 클래스를 맞추는 작업을 통해 오차를 계산해 Gradient Descent 방법을 통해 학습
    - LeNet-5 이전의 패턴 인식 구조에서 Feature Extraction 작업은 CNN 구조를 통해 대체될 수 있고
    - Classifier Module은 Fully connected layer를 통해 대체될 수 있음
    - 이때 CNN과 Fully Connected layer이 가능하게 된 배경은 빠른 알고리즘과 적은 비용으로도 높은 성능의 컴퓨터를 사용할 수 있게 되면서 연산량이 많은 CNN과 Fully Connected layer를 사용할 수 있게 되었음

In [6]:
import numpy as np
import pandas as pd

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.transforms import Compose, ToTensor

In [8]:
transforms = Compose([
   ToTensor(),
])

In [9]:
train = torchvision.datasets.MNIST('data', train=True, download=True, transform=transforms)
test = torchvision.datasets.MNIST('data', train=False, download=True, transform=transforms)
train_loader = torch.utils.data.DataLoader(train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=100, shuffle=False)

### M.L.P(Baseline Model with Multilayer Perceptrons)

In [11]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(784, 784)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(784, 10)
        
    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.act1(self.layer1(x))
        x = self.layer2(x)
        return x

In [12]:
model = MLP()
optimizer = optim.SGD(model.parameters(), lr=0.01)
# optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

n_epochs = 5
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    acc = 0
    count = 0
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        acc += (torch.argmax(y_pred, 1) == y_batch).float().sum()
        count += len(y_batch)
    acc = acc / count
    print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))

Epoch 0: model accuracy 84.93%
Epoch 1: model accuracy 88.31%
Epoch 2: model accuracy 89.64%
Epoch 3: model accuracy 90.02%
Epoch 4: model accuracy 90.67%


### Simple Convolutional Neural Network for MNIST

### 합성곱
- 커널 : 이미지로부터 특징을 추출하기 위한 가중치 행렬
- 특징 : 합성곱의 결과로부터 얻어지는 이미지
- stride : 커널이 움직이는 거리

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 10, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)
        self.dropout = nn.Dropout(0.2)
        self.flat = nn.Flatten()
        self.fc = nn.Linear(27*27*10, 128)
        self.relu2 = nn.ReLU()
        self.output = nn.Linear(128, 10)
        
    def forward(self, x):
        x = self.relu1(self.conv(x))
        x = self.pool(x)
        x = self.dropout(x)
        x = self.relu2(self.fc(self.flat(x)))
        x = self.output(x)
        return x

In [6]:
model = CNN()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

n_epochs = 5
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    model.eval()
    acc = 0
    count = 0
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        acc += (torch.argmax(y_pred, 1) == y_batch).float().sum()
        count += len(y_batch)
    acc = acc / count
    print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))

Epoch 0: model accuracy 97.43%
Epoch 1: model accuracy 98.01%
Epoch 2: model accuracy 97.72%
Epoch 3: model accuracy 98.06%
Epoch 4: model accuracy 98.17%


## LeNet-5 구현

### LeNet-5 for MNIST

In [7]:
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.act1 = nn.Tanh()
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.act2 = nn.Tanh()
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0)
        self.act3 = nn.Tanh()

        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(1*1*120, 84)
        self.act4 = nn.Tanh()
        self.fc2 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.act1(self.conv1(x))
        x = self.pool1(x)
        x = self.act2(self.conv2(x))
        x = self.pool2(x)
        x = self.act3(self.conv3(x))
        x = self.act4(self.fc1(self.flat(x)))
        x = self.fc2(x)
        return x

In [8]:
model = LeNet5()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

n_epochs = 5
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    model.eval()
    acc = 0
    count = 0
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        acc += (torch.argmax(y_pred, 1) == y_batch).float().sum()
        count += len(y_batch)
    acc = acc / count
    print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))

Epoch 0: model accuracy 95.43%
Epoch 1: model accuracy 97.16%
Epoch 2: model accuracy 97.77%
Epoch 3: model accuracy 97.76%
Epoch 4: model accuracy 98.44%


In [9]:
torch.save(model.state_dict(), "LeNet5_20240129.pth") # 모델 저장

## CNN 모델 파라매터 확인 실습

In [58]:
# 임의의 텐서를 강제로 생성
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


In [59]:
conv1 = nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2)
print(conv1)

Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))


In [60]:
conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [61]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [62]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [63]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [64]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [65]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [66]:
print(out.size(0), out.size(1), out.size(2), out.size(3))

1 64 7 7


In [68]:
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
out = out.view(out.size(0), -1) 
print(out.shape)

torch.Size([1, 3136])


In [69]:
fc = nn.Linear(3136, 10) # input_dim = 3,136, output_dim = 10
out = fc(out)
print(out.shape)

torch.Size([1, 10])


### View와 Reshape

In [2]:
import torch
x = torch.arange(12)
print(x) # tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])


In [3]:
print(x.reshape(2, 6))
print(x.view(2, 6))

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])
tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


In [4]:
print(x.reshape(2, 3, -1)) # (2, 3, 2) 차원으로 자동 지정
print(x.view(2, 2, -1)) # (2, 2, 3) 차원으로 자동 지정

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]]])
tensor([[[ 0,  1,  2],
         [ 3,  4,  5]],

        [[ 6,  7,  8],
         [ 9, 10, 11]]])


torch.view와 torch.reshape의 가장 큰 차이는 contiguous 속성을 만족하지 않는 텐서에 적용이 가능하느냐 여부입니다.

In [8]:
a = torch.randn(3, 4)
a.transpose_(0, 1)
b = torch.randn(4, 3)
print(a)
print(b)

tensor([[ 0.2699,  0.2880,  0.0943],
        [ 0.2418,  1.3825,  0.3678],
        [ 0.2878,  0.4216, -0.3238],
        [ 0.8696,  0.7349, -0.2613]])
tensor([[-0.0263, -2.1660, -0.6443],
        [-0.5453, -1.3778, -1.1625],
        [-0.9622, -0.5388,  0.1624],
        [-0.0874, -0.0213,  0.4618]])


In [10]:
# a 텐서 메모리 주소 예시
for i in range(4):
    for j in range(3):
        print("a :", a[i][j].data_ptr())
# b 텐서 메모리 주소 예시
for i in range(4):
    for j in range(3):
        print("b :", b[i][j].data_ptr())

a : 3044972495872
a : 3044972495888
a : 3044972495904
a : 3044972495876
a : 3044972495892
a : 3044972495908
a : 3044972495880
a : 3044972495896
a : 3044972495912
a : 3044972495884
a : 3044972495900
a : 3044972495916
b : 3044972496000
b : 3044972496004
b : 3044972496008
b : 3044972496012
b : 3044972496016
b : 3044972496020
b : 3044972496024
b : 3044972496028
b : 3044972496032
b : 3044972496036
b : 3044972496040
b : 3044972496044


b는 axis = 0인 오른쪽 방향으로 자료가 순서대로 저장됨에 비해, a는 transpose 연산을 거치며 axis = 1인 아래 방향으로 자료가 저장되고 있었습니다. 여기서, b처럼 axis 순서대로 자료가 저장된 상태를 contiguous = True 상태라고 부르며, a같이 자료 저장 순서가 원래 방향과 어긋난 경우를 contiguous = False 상태라고 합니다.

텐서의 shape을 조작하는 과정에서 메모리 저장 상태가 변경되는 경우가 있습니다. 주로 narrow(), view(), expand(), transpose() 등 메소드를 사용하는 경우에 이 상태가 깨지는 것으로 알려져 있습니다.

In [11]:
y = torch.ones(3, 4)
y.transpose_(0, 1)
y.is_contiguous() # False

False

In [12]:
print(y.reshape(3, 2, 2)) # 실행 가능
# print(y.view(3, 2, 2)) # 실행 불가

tensor([[[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]]])


```
RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.
```

따라서, 차원 변환을 적용하려는 텐서의 상태에 대하여 정확하게 파악하기가 모호한 경우에는 view 대신 reshape를 사용하시는 것을 권장드립니다.

In [13]:
#!pip install torchinfo

In [14]:
from torchinfo import summary
summary(model, input_size=(1, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
MLP                                      [1, 10]                   --
├─Linear: 1-1                            [1, 784]                  615,440
├─ReLU: 1-2                              [1, 784]                  --
├─Linear: 1-3                            [1, 10]                   7,850
Total params: 623,290
Trainable params: 623,290
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.62
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 2.49
Estimated Total Size (MB): 2.50

In [16]:
import torch.nn.functional as F

def findConv2dOutShape(H_in, W_in, conv, pool=2):
    kernel_size = conv.kernel_size
    stride = conv.stride
    padding = conv.padding
    dilation = conv.dilation

    H_out = np.floor((H_in + 2*padding[0] - dilation[0]*(kernel_size[0]-1)-1) / stride[0] + 1)
    W_out = np.floor((W_in + 2*padding[1] - dilation[1]*(kernel_size[1]-1)-1) / stride[1] + 1)

    if pool:
        H_out /= pool
        W_out /= pool
    
    return int(H_out), int(W_out)

class Net(nn.Module):
    def __init__(self, params):
        super(Net, self).__init__()
        C_in, H_in, W_in = params['input_shape']
        init_f = params['initial_filters']
        num_fc1 = params['num_fc1']
        num_classes = params['num_classes']
        self.dropout_rate = params['dropout_rate']

        self.conv1 = nn.Conv2d(C_in, init_f, kernel_size=3)
        h,w = findConv2dOutShape(H_in, W_in, self.conv1)
        self.conv2 = nn.Conv2d(init_f, 2*init_f, kernel_size=3)
        h,w = findConv2dOutShape(h, w, self.conv2)
        self.conv3 = nn.Conv2d(2*init_f, 4*init_f, kernel_size=3)
        h,w = findConv2dOutShape(h, w, self.conv3)
        self.conv4 = nn.Conv2d(4*init_f, 8*init_f, kernel_size=3)
        h,w = findConv2dOutShape(h, w, self.conv4)

        self.num_flatten = h*w*8*init_f
        self.fc1 = nn.Linear(self.num_flatten, num_fc1)
        self.fc2 = nn.Linear(num_fc1, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, self.num_flatten)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, self.dropout_rate, training = self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)